In [1]:
import numpy as np
import torch
import copy
from metabci.brainda.algorithms.deep_learning import ShallowNet_Sparse
from metabci.brainda.algorithms.utils.model_selection import (
    set_random_seeds,
    generate_kfold_indices, match_kfold_indices)
from metabci.brainda.datasets import Wang2016, BETA
# 三个SSVEP Datasets：Nakanishi2015、Wang2016、BETA
from metabci.brainda.paradigms import SSVEP
from skorch.helper import predefined_split

In [2]:
#**************************************************
# Benchmark数据集读取处理
#**************************************************
Bench_dataset = Wang2016()
subject_list = list(range(1, 21))  # 被试编号从1到20
for s in subject_list:
    Bench_dataset.data_path(subject=s, path="E:\\MetaBCI-master\\mne_data")  # 依次为每个被试设置路径
events = Bench_dataset.events.keys()
freq_list = [str(Bench_dataset.get_freq(event)) for event in events]  # 获得所有刺激的频率
freq_map = {i: freq for i, freq in enumerate(freq_list)}  # 标签到频率的映射

Bench_subjects = list(range(1, 21))  # 使用S1–S20被试进行训练
Bench_paradigm = SSVEP(
    channels=['POZ', 'PZ', 'PO3', 'PO5', 'PO4', 'PO6', 'O1', 'OZ', 'O2'],  # 选择电极通道
    intervals=[(0.14, 1.14)],  # 分析时间窗最长-0.5-5.5s ,0.0是刺激开始
    events=freq_list,  # 选择所有刺激频率
    srate=250  # 采样率
)

# add 5-90Hz bandpass filter in raw hook
# 对原始EEG信号做5-90Hz带通滤波。
def raw_hook(raw, caches):
    raw.filter(5, 90, l_trans_bandwidth=2, h_trans_bandwidth=5, phase='zero-double')
    caches['raw_stage'] = caches.get('raw_stage', -1) + 1
    return raw, caches

Bench_paradigm.register_raw_hook(raw_hook)



# 获取微调数据
X_Bench, y_Bench, meta_Bench = Bench_paradigm.get_data(
    Bench_dataset,
    subjects=Bench_subjects,
    return_concat=True,
    n_jobs=None,
    verbose=False
)


--------ssssss, /upload/yijun/S1.mat.7z
--------ssssss, /upload/yijun/S2.mat.7z
--------ssssss, /upload/yijun/S3.mat.7z
--------ssssss, /upload/yijun/S4.mat.7z
--------ssssss, /upload/yijun/S5.mat.7z
--------ssssss, /upload/yijun/S6.mat.7z
--------ssssss, /upload/yijun/S7.mat.7z
--------ssssss, /upload/yijun/S8.mat.7z
--------ssssss, /upload/yijun/S9.mat.7z
--------ssssss, /upload/yijun/S10.mat.7z
--------ssssss, /upload/yijun/S11.mat.7z
--------ssssss, /upload/yijun/S12.mat.7z
--------ssssss, /upload/yijun/S13.mat.7z
--------ssssss, /upload/yijun/S14.mat.7z
--------ssssss, /upload/yijun/S15.mat.7z
--------ssssss, /upload/yijun/S16.mat.7z
--------ssssss, /upload/yijun/S17.mat.7z
--------ssssss, /upload/yijun/S18.mat.7z
--------ssssss, /upload/yijun/S19.mat.7z
--------ssssss, /upload/yijun/S20.mat.7z
--------ssssss, /upload/yijun/S1.mat.7z
--------ssssss, /upload/yijun/S2.mat.7z
--------ssssss, /upload/yijun/S3.mat.7z
--------ssssss, /upload/yijun/S4.mat.7z
--------ssssss, /upload/yijun

In [3]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_eegnet.py # Bench数据集 S1~S20 学习率1e-4，F1=40,D=2,F2=80
# 所有折准确率: [0.87375, 0.88625, 0.87625, 0.87875, 0.85125, 0.8875]
# 平均准确率： 0.875625                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
shared_ratio = 0.2
alpha = 1
epsilon = 60
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4

Bench_save_path = "Bench_shallownet_mask_1s_60_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
    estimator.set_params(optimizer__lr=1e-2,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 计算验证集准确率
    val_preds = estimator.predict(X_Bench[new_valid_ind])
    val_acc = np.mean(val_preds == y_Bench[new_valid_ind])
    #print("当前折验证集准确率：", val_acc)
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
in:825
out:40
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0265        4.8673       0.0338        4.6185     +  0.0100  1.0938
      2       0.0850        4.0617       0.1163        4.2182     +  0.0100  0.8669
      3       0.2450        3.0815       0.3812        2.8090     +  0.0100  0.8369
      4       0.4798        2.0140       0.5175        2.2026     +  0.0100  0.8507
      5       0.5773        1.6591       0.5150        2.1947     +  0.0100  0.8510
      6       0.6008        1.5283       0.5337        2.3556        0.0100  0.8538
      7       0.6388        1.3700       0.5500        2.3476        0.0100  0.8536
      8       0.6600        1.2718       0.5587        2.3419        0.0100  0.8374
      9       0.6673        1.2693       0.5613        2.4879        0

In [4]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_eegnet.py # Bench数据集 S1~S20 学习率1e-4，F1=40,D=2,F2=80
# 所有折准确率: [0.87375, 0.88625, 0.87625, 0.87875, 0.85125, 0.8875]
# 平均准确率： 0.875625                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
shared_ratio = 0.2
alpha = 1
epsilon = 70
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4

Bench_save_path = "Bench_shallownet_mask_1s_70_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
    estimator.set_params(optimizer__lr=1e-2,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 计算验证集准确率
    val_preds = estimator.predict(X_Bench[new_valid_ind])
    val_acc = np.mean(val_preds == y_Bench[new_valid_ind])
    #print("当前折验证集准确率：", val_acc)
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
in:825
out:40
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0278        4.5992       0.0400        4.7212     +  0.0100  0.8584
      2       0.0902        3.8999       0.1275        4.0912     +  0.0100  0.8624
      3       0.2938        2.7891       0.4062        2.7080     +  0.0100  0.8794
      4       0.5240        1.8461       0.4975        2.3116     +  0.0100  0.8537
      5       0.6008        1.5415       0.5425        2.2782     +  0.0100  0.8500
      6       0.6222        1.4224       0.5413        2.6167        0.0100  0.8582
      7       0.6490        1.2800       0.5437        2.4397        0.0100  0.8710
      8       0.6773        1.2152       0.5825        2.4532        0.0100  0.8708
      9       0.6810        1.1457       0.5550        2.6128        0

In [5]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_eegnet.py # Bench数据集 S1~S20 学习率1e-4，F1=40,D=2,F2=80
# 所有折准确率: [0.87375, 0.88625, 0.87625, 0.87875, 0.85125, 0.8875]
# 平均准确率： 0.875625                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
shared_ratio = 0.2
alpha = 1
epsilon = 80
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4

Bench_save_path = "Bench_shallownet_mask_1s_80_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
    estimator.set_params(optimizer__lr=1e-2,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 计算验证集准确率
    val_preds = estimator.predict(X_Bench[new_valid_ind])
    val_acc = np.mean(val_preds == y_Bench[new_valid_ind])
    #print("当前折验证集准确率：", val_acc)
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
in:825
out:40
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0255        4.4195       0.0362        4.7250     +  0.0100  0.8341
      2       0.0990        3.7535       0.1525        3.9519     +  0.0100  0.8456
      3       0.3380        2.5853       0.4525        2.5730     +  0.0100  0.8460
      4       0.5485        1.7426       0.5138        2.5231     +  0.0100  0.8750
      5       0.5958        1.5295       0.5537        2.5044     +  0.0100  0.8415
      6       0.6188        1.3959       0.5513        2.8302        0.0100  0.8375
      7       0.6480        1.2855       0.5475        2.7286        0.0100  0.8561
      8       0.6677        1.2014       0.5637        2.8468        0.0100  0.8553
      9       0.6735        1.1528       0.5288        3.1147        0

In [6]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_eegnet.py # Bench数据集 S1~S20 学习率1e-4，F1=40,D=2,F2=80
# 所有折准确率: [0.87375, 0.88625, 0.87625, 0.87875, 0.85125, 0.8875]
# 平均准确率： 0.875625                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
shared_ratio = 0.2
alpha = 1
epsilon = 90
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4

Bench_save_path = "Bench_shallownet_mask_1s_90_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
    estimator.set_params(optimizer__lr=1e-2,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 计算验证集准确率
    val_preds = estimator.predict(X_Bench[new_valid_ind])
    val_acc = np.mean(val_preds == y_Bench[new_valid_ind])
    #print("当前折验证集准确率：", val_acc)
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
in:825
out:40
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0255        4.3137       0.0475        4.8546     +  0.0100  0.8629
      2       0.0975        3.7117       0.1500        4.1026     +  0.0100  0.8542
      3       0.3257        2.6334       0.4338        2.7943     +  0.0100  0.8401
      4       0.5292        1.7789       0.5088        2.6966     +  0.0100  0.8459
      5       0.6020        1.4872       0.5450        2.6263     +  0.0100  0.8476
      6       0.6418        1.3460       0.5550        2.8399        0.0100  0.8625
      7       0.6635        1.2284       0.5475        2.8450        0.0100  0.8351
      8       0.6905        1.1382       0.5625        2.7859        0.0100  0.8419
      9       0.6973        1.0914       0.5650        3.0591        0

In [7]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_eegnet.py # Bench数据集 S1~S20 学习率1e-4，F1=40,D=2,F2=80
# 所有折准确率: [0.87375, 0.88625, 0.87625, 0.87875, 0.85125, 0.8875]
# 平均准确率： 0.875625                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
shared_ratio = 0.2
alpha = 1
epsilon = 100
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4

Bench_save_path = "Bench_shallownet_mask_1s_100_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
    estimator.set_params(optimizer__lr=1e-2,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 计算验证集准确率
    val_preds = estimator.predict(X_Bench[new_valid_ind])
    val_acc = np.mean(val_preds == y_Bench[new_valid_ind])
    #print("当前折验证集准确率：", val_acc)
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
in:825
out:40
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0222        4.2195       0.0550        4.9195     +  0.0100  0.8508
      2       0.0948        3.6347       0.1600        4.2688     +  0.0100  0.8652
      3       0.3350        2.5793       0.4425        2.9532     +  0.0100  0.8709
      4       0.5450        1.7236       0.5262        2.7840     +  0.0100  0.8341
      5       0.6192        1.4262       0.5550        2.7520     +  0.0100  0.8324
      6       0.6515        1.2736       0.5575        2.8535        0.0100  0.8476
      7       0.6740        1.1762       0.5675        2.7707        0.0100  0.8625
      8       0.6975        1.1060       0.5913        2.9079        0.0100  0.8641
      9       0.6995        1.0383       0.5962        3.0180        0

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35]  # 测试使用 
pre_save_path = "Bench_shallownet_mask_1s_60_0.2.pth"
# 加载预训练模型

n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
shared_ratio = 0.2
alpha = 1
epsilon = 60
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4

estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
in:825
out:40
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 75.4%
subject 24 的准确率: 69.2%
subject 27 的准确率: 72.9%
subject 28 的准确率: 77.9%
subject 31 的准确率: 76.7%
subject 34 的准确率: 66.2%
subject 35 的准确率: 83.8%
所有被试的准确率： {'22': 0.7541666666666667, '24': 0.6916666666666667, '27': 0.7291666666666666, '28': 0.7791666666666667, '31': 0.7666666666666667, '34': 0.6625, '35': 0.8375}
所有被试的平均准确率: 74.6%
所有被试准确率（无名称）: 75.4,69.2,72.9,77.9,76.7,66.2,83.8


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35]  # 测试使用 
pre_save_path = "Bench_shallownet_mask_1s_70_0.2.pth"
# 加载预训练模型

n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
shared_ratio = 0.2
alpha = 1
epsilon =70
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4

estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
in:825
out:40
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 79.6%
subject 24 的准确率: 78.8%
subject 27 的准确率: 74.2%
subject 28 的准确率: 76.2%
subject 31 的准确率: 75.4%
subject 34 的准确率: 70.0%
subject 35 的准确率: 82.9%
所有被试的准确率： {'22': 0.7958333333333333, '24': 0.7875, '27': 0.7416666666666667, '28': 0.7625, '31': 0.7541666666666667, '34': 0.7, '35': 0.8291666666666667}
所有被试的平均准确率: 76.7%
所有被试准确率（无名称）: 79.6,78.8,74.2,76.2,75.4,70.0,82.9


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35]  # 测试使用 
pre_save_path = "Bench_shallownet_mask_1s_80_0.2.pth"
# 加载预训练模型

n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
shared_ratio = 0.2
alpha = 1
epsilon = 80
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4

estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
in:825
out:40
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 76.2%
subject 24 的准确率: 75.0%
subject 27 的准确率: 72.9%
subject 28 的准确率: 77.1%
subject 31 的准确率: 77.5%
subject 34 的准确率: 71.2%
subject 35 的准确率: 81.7%
所有被试的准确率： {'22': 0.7625, '24': 0.75, '27': 0.7291666666666666, '28': 0.7708333333333334, '31': 0.775, '34': 0.7125, '35': 0.8166666666666667}
所有被试的平均准确率: 75.9%
所有被试准确率（无名称）: 76.2,75.0,72.9,77.1,77.5,71.2,81.7


In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35]  # 测试使用 
pre_save_path = "Bench_shallownet_mask_1s_90_0.2.pth"
# 加载预训练模型

n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
shared_ratio = 0.2
alpha = 1
epsilon = 90
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4

estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
in:825
out:40
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 78.3%
subject 24 的准确率: 75.8%
subject 27 的准确率: 71.7%
subject 28 的准确率: 74.2%
subject 31 的准确率: 67.9%
subject 34 的准确率: 67.1%
subject 35 的准确率: 84.2%
所有被试的准确率： {'22': 0.7833333333333333, '24': 0.7583333333333333, '27': 0.7166666666666667, '28': 0.7416666666666667, '31': 0.6791666666666667, '34': 0.6708333333333333, '35': 0.8416666666666667}
所有被试的平均准确率: 74.2%
所有被试准确率（无名称）: 78.3,75.8,71.7,74.2,67.9,67.1,84.2


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35]  # 测试使用 
pre_save_path = "Bench_shallownet_mask_1s_100_0.2.pth"
# 加载预训练模型

n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
shared_ratio = 0.2
alpha = 1
epsilon = 100
n_time_filters = 20
time_kernel = 25
n_space_filters = 15
pool_kernel = 8
pool_stride = 4

estimator = ShallowNet_Sparse(n_time_filters, time_kernel,n_space_filters,pool_kernel, pool_stride, n_channels, n_samples, n_classes, shared_ratio, alpha, epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
in:825
out:40
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 83.3%
subject 24 的准确率: 76.7%
subject 27 的准确率: 77.9%
subject 28 的准确率: 78.3%
subject 31 的准确率: 79.2%
subject 34 的准确率: 70.4%
subject 35 的准确率: 83.8%
所有被试的准确率： {'22': 0.8333333333333334, '24': 0.7666666666666667, '27': 0.7791666666666667, '28': 0.7833333333333333, '31': 0.7916666666666666, '34': 0.7041666666666667, '35': 0.8375}
所有被试的平均准确率: 78.5%
所有被试准确率（无名称）: 83.3,76.7,77.9,78.3,79.2,70.4,83.8
